In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/05 15:13:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2

In [3]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [4]:
from datetime import datetime

In [5]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [6]:
rows = rdd.take(10)
row = rows[0]
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 12, 18, 15, 4), PULocationID=41, total_amount=7.88)

In [7]:
def mapper_function(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [8]:
rdd \
    .filter(filter_outliers) \
    .map(mapper_function) \
    .take(10)

[((datetime.datetime(2020, 1, 12, 18, 0), 41), (7.88, 1)),
 ((datetime.datetime(2020, 1, 31, 20, 0), 173), (8.3, 1)),
 ((datetime.datetime(2020, 1, 7, 8, 0), 74), (23.46, 1)),
 ((datetime.datetime(2020, 1, 15, 14, 0), 25), (7.3, 1)),
 ((datetime.datetime(2020, 1, 31, 10, 0), 259), (25.54, 1)),
 ((datetime.datetime(2020, 1, 18, 17, 0), 177), (13.3, 1)),
 ((datetime.datetime(2020, 1, 17, 20, 0), 65), (11.16, 1)),
 ((datetime.datetime(2020, 1, 13, 10, 0), 165), (20.56, 1)),
 ((datetime.datetime(2020, 1, 7, 15, 0), 170), (49.05, 1)),
 ((datetime.datetime(2020, 1, 10, 11, 0), 74), (9.56, 1))]

In [9]:
def reducer_function(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [10]:
rdd \
    .filter(filter_outliers) \
    .map(mapper_function) \
    .reduceByKey(reducer_function) \
    .take(10)

[((datetime.datetime(2020, 1, 8, 20, 0), 75), (932.3999999999996, 66)),
 ((datetime.datetime(2020, 1, 16, 18, 0), 41), (591.8100000000001, 54)),
 ((datetime.datetime(2020, 1, 23, 10, 0), 43), (315.40000000000003, 19)),
 ((datetime.datetime(2020, 1, 8, 16, 0), 74), (1398.779999999999, 74)),
 ((datetime.datetime(2020, 1, 21, 8, 0), 17), (90.17999999999999, 5)),
 ((datetime.datetime(2020, 1, 28, 8, 0), 121), (45.11, 2)),
 ((datetime.datetime(2020, 1, 4, 15, 0), 25), (457.37000000000006, 25)),
 ((datetime.datetime(2020, 1, 2, 16, 0), 166), (555.48, 32)),
 ((datetime.datetime(2020, 1, 24, 1, 0), 129), (155.73000000000002, 12)),
 ((datetime.datetime(2020, 1, 18, 19, 0), 82), (356.60000000000014, 28))]

In [11]:
from collections import namedtuple

In [12]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [13]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [14]:
from pyspark.sql import types

In [15]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [17]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(mapper_function) \
    .reduceByKey(reducer_function) \
    .map(unwrap) \
    .toDF(result_schema)

In [18]:
 df_result.show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-08 20:00:00|  75| 932.3999999999996|   66|
|2020-01-16 18:00:00|  41| 591.8100000000001|   54|
|2020-01-23 10:00:00|  43|315.40000000000003|   19|
|2020-01-08 16:00:00|  74| 1398.779999999999|   74|
|2020-01-21 08:00:00|  17| 90.17999999999999|    5|
|2020-01-28 08:00:00| 121|             45.11|    2|
|2020-01-04 15:00:00|  25|457.37000000000006|   25|
|2020-01-02 16:00:00| 166|            555.48|   32|
|2020-01-24 01:00:00| 129|155.73000000000002|   12|
|2020-01-18 19:00:00|  82|356.60000000000014|   28|
|2020-01-12 12:00:00| 254|             81.74|    4|
|2020-01-12 15:00:00|  74|1287.1899999999991|   85|
|2020-01-24 18:00:00| 130|            546.78|   29|
|2020-01-08 15:00:00|  95|            441.29|   39|
|2020-01-07 06:00:00|  95|             85.64|    3|
|2020-01-07 19:00:00| 238|             63.31|    2|
|2020-01-09 

In [19]:
df_result.write.parquet('tmp/green-revenue-rdd')

In [21]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [39]:
import pandas as pd

In [46]:
# Dummy model to predict how long the trip will take
def model_predict(df):
    #y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [47]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [48]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [49]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|3.9000000000000004|
|               4.9|
|              13.5|
|               4.0|
|             11.65|
|13.100000000000001|
|5.6499999999999995|
| 6.800000000000001|
|             55.75|
|               8.9|
|               5.0|
|             13.75|
|               5.5|
|             19.05|
|              9.25|
|              45.7|
|               5.2|
| 5.699999999999999|
|              5.75|
|4.6000000000000005|
+------------------+
only showing top 20 rows



23/03/06 07:44:13 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 34895468 ms exceeds timeout 120000 ms
23/03/06 07:44:14 WARN SparkContext: Killing executors is not supported by current scheduler.
